# Processing of holograms

## 1. Calibration by window inspection

### Acquisition

Get several background images to reduce noise by averaging. The python script `Mean_Image.py` may be edited and used directly in the acquisition PC to get a mean image. This script is part of the GitHub project `ZaberAndEuresys`.

### Pre-processing

Use functions in `C:\Work\Scripting` folder. This folder is repeated in `C:\Users\j05625pe\Documents\Work\JASMIN\Projects\Single ASM` and placed in a GitHub repository. There are some extra files in the latter that are bash scripts and mcc compiled versions for running in JASMIN.

Use `PrepImHoloviewer.m` to equalize images and to get an average image from several images with same name but ending with number of image in a sequence, eg Images00.tif, Images01.tif, .... Use `image_path = 'path\Images**.tif';`.

Send resulting images to JASMIN. (See using JASMIN.ipynb)

### Prepare distances to propagate in slurm-bash script

Use a config file for calculating autofocusing methods at a range of z positions. Example:
> % Configuration file for Single_ASM.m <br>
**image_path ./holograms/First_Levitation/Background_aver.png** <br>
**output_path ./results** <br>
bgnd_path [] % Could be used for dividing the input image by a background image <br>
zeval [] % Positions where an output image is wanted <br>
% zeval 20.2e-3 29.04e-3 39.56e-3 42.9e-3 50.7e-3 54.14e-3 57.93e-3 61.54e-3 63.27e-3 67.35e-3 68.38e-3 <br>
downsampling 1 <br>
crop_factor 1 <br>
**z_autofoc 15e-3 70e-3 5000**	% In terms of effective distance (divergent beam) (5.48mm under L) <br>
**L 76e-3** <br>
dx 2.2e-6 <br>
dy 2.2e-6 <br>
wavelength 355e-9 <br>
make_movie false <br>
windows 1.4761 5e-3 1.4761 12e-3 <br>
size_windows 7 160 % For focusing methods <br>
sub_mean_im true % subtract the mean intensity <br>
auto_min_max_video true <br>
use_equiv_wavel false % Use an equivalent wavelength instead of equivalent distances

Run with an array of processors with a file `run_array_slurm.sh`:
> #!/bin/bash  <br>
#SBATCH --partition=short-serial <br>
#SBATCH -o ./status/%A_%a.out  <br>
#SBATCH -e ./status/%A_%a.err <br>
#SBATCH --mem=20G <br>
#SBATCH -t 4:00:00 <br>
#SBATCH --array=1-1200 <br>
 <br>
`./run_Single_ASM_j.sh ~/MCR/MATLAB_Runtime_2019a/v96 Config_Divergent.txt $SLURM_ARRAY_TASK_ID $SLURM_ARRAY_TASK_MAX` <br>

### Run and join results from different processors

Run script by using `sbatch run_array_slurm.sh`. Use `ls | wc -l; date +"%T"` in status/results folder to check how many processes are working. When the processing ends, check status of errors and outputs, or just update `run_join_I_MCR.sh` with the proper results folder (see config file) and run. If everything went okay with the reconstruction, then the small files will be deleted and joined up in a single output file. Otherwise, update the script `run_job_errored` with the number of required processors in the array, the original size of the array and the used configuration file. This will reattempt to get the proper outputs.

Copy the output to the local PC (`C:\Work\Scripting\results`) and rewrite name of file to somthing relevant. Use `analysis_Indices.m` to get probable values of positions of windows (2 plots for max/min index values and averaged power of values when scanning images). Holoviewer can be used afterwards to see if these are distances where dust particles on the windows are seen in focus. Use scripts `z_spherical2planar.m` to get the distances to evaluate in Holoviewer. Once a list of planar distances of interest is obtained, use `z_planar2spherical.m` to get the spherical source (physical) distances.

### End calibration

Do the calibration by estimating the distance between punctual source and sensor. Use `C:\Work\Scripting\Estimate_L_bgnd_windows.m`. Use the same L_guess that was used in the configuration file. Repeat same values of properties of windows. If zs has only 2 values, the sensor-side window is considered. If 4 values are used, the position of both windows are estimated along with the distance from the sensor to the source.

## 2. Calibration by periodic sample

Use python code for getting multiple images of a sample with a periodic grid (Thorlabs R2L2S3P1) at different distances, moved with motorised linear stage. Then use `C:\Work\Scripting\focus_to_camera_estimation_long_set.m`.

If the images are obtained by manual traslation, then use `C:\Work\Scripting\focus_to_camera_estimation_variable_steps.m`. This script reads the distances for each image from a text file.

## 3. Processing of single hologram with Holosuite

Documentation is being added to the file `axial_sampling_doc.m` in branch Pablo_feature_zs for changes in HoloSuite files.

### Compiling issues solved

- Statistics toolbox missing in Linux distro:
    - Use lsqcurvefit instead of nlinfit.
    - Use alt_stats as an alternative to the toolbox.
    - Create range : @(x) max(x) - min(x)
- lowpass, highpass,.. resources : mcc -v -m HolosuiteReconstructSingleHolo.m -a bkgndMedDiv.m forceMeanAndSTD.m highPass.m imDetrend.m imnegative.m localMedDiv.m localMedSub.m lowPass.m

### Config file

Use `C:\Work\Scripting\CompileHolosuite\Results` to keep record of all used config files (Linux and JASMIN). The results coming from the processing should be saved here too. Use the slurm scripts `C:\Work\Scripting\CompileHolosuite\slurm_HRSH.sh` and `slurm_array_HRSH.sh` to define the location of the config file and the name of the hologram file. Also set `path` of the hologram in config file (`current_hologram` is not necessary) and result folder in `localTmp`.

### Run in Windows

Use `C:\Work\Scripting\useHolosuiteOneHologram.m`. Check paths and hologram file name.

### Run in Linux VM

Use `/home/pablo/Desktop/InHost/C_Work/Scripting/CompileHolosuite/HolosuiteReconstructSingleHolo.m`, or its compilation with bash script `HRSH.sh`. Check paths and hologram file name. See `debug_joinJobs.m` for a complete procedure (with missing reconstruction files).

### Run in JASMIN

#### Images

Upload images with rsync or using scp with Mobaxterm bash script in `PC-W:send_dirs_scp.sh`.

#### Single job

Do `sbatch slurm_HRSH.sh`. Save the output in the corresponding folders in JASMIN and `C:\Work\Scripting\CompileHolosuite` by using git.

#### Job array

1. Prepare script `slurm_array_HRSH.sh` with the required amount of jobs. Do `sbatch slurm_array_HRSH.sh`.
2. When all the jobs are completed run from scientific machine: `./JASMIN_JoinAndHist.sh` (the config file and hologram should be the same as in `slurm_array_HRSH.sh`.
3. If there are missing files, edit `slurm_array_HRSH.sh` as explained in `missing_jobs_files.txt` (USING_JOB_ID and number of jobs in array). Run `sbatch slurm_array_HRSH.sh`.
4. Repeat 2 and 3 until the hist file is created.

#### Git

Git: PablitoE : Token : 002d79a2e0f6045365097145d1eb374c3731d4bb

Use git to upload the config files and tree directory of Results.

## 4. Processing a sequence of holograms with Holosuite

First read 3.

### Config file

`path` needs the directory where the holograms are. `localTmp` is where the config file is, the results are saved there.

Add lines for the sequences to process with format `first_image_name[:step]:last_image_name,output_folder_name`.

For using images from the sequence as background, use one of the following as first filter input in the config file:

```
bkgndMedDiv : amplitude
  NBkgndFiles = 7
  fileskip = 1
  holoFilenameList = _currentSeqList
  currentHoloFilename = _current_holo
  holoPath = _path
  
bkgndHoloDiv : amplitude
  BgndHoloFile = path\to\bgnd\file
  currentHoloFilename = _current_holo
```
  
Normally, the images will be already contrast images creaated by using Python scripts as `contrast_video.py`.

### Setting the config file: rules and threshold

A single image should be observed with holoViewer to check if the threshold value is okay. Also check filling gaps and expansion, and spatial filters cutoffs. A set of config files for multiple sequences can be generated using `set_configs_seqs.m`. Once that is set in the config file, run the reconstruction over a single sequence, or part of it. Use `carft.m` to find rules and the tree that would classify particles as particles and artifacts as artifacts. It is good practice to start with a rule on `numzs` or `tphunderthr`, classify some of each class, predict and then see the tree to start challenging the decisions of the tree.

### Reconstruction of a sequence of holograms using Holosuite in local PC

The images in a sequence could be equalised (self or using background images) and converted to PNG format, use `PrepSeqHoloSuite`. It is better to perform the equalisation by extracting a background hologram from the images. To easily convert images to PNG use `mogrify -format png -path ./new_folder  *.tiff`. A better option is to use scripts such as `contrast_video.py` or `contrast_video_from_images.py`. The background is obtained by removing outliers from the calculation of the median intensity. If the contrast holograms are found by doing $\frac{I-I_B}{I_B}$, the threshold should be set as high and low, but for $\frac{I}{I_B}$, using the lower threshold is enough. The contrast holograms are saved along a `config.txt` that says the scaling factors. Holosuite can read the images with these factors so that the dynamic range is kept reasonably good.

They are then reconstructed using `useHolosuiteSequence.m`, where the config file must be set.

### Run in JASMIN

#### Single job

Use `slurm_HRSeq.sh`. Save the output in the corresponding folders in JASMIN and `C:\Work\Scripting\CompileHolosuite` by using rsync. 

#### Single or multiple sequences of holograms with SLURM array

Use `slurm_array_HRSeqs.sh`. Set the amount of jobs (as high as the amount of holograms in all the sequences). All the sequences should be in their own directory (with a config file) within a single main directory. Adapt `reconSomeSequences.py` with this information (MAIN_PATH). Config files should have the name given in the Python script in different folders, inside the directories of list MAIN_PATHS, or just a single config file in the list.

#### Check output for errors

Open Jupyter Notebook in JASMIN `Check_output_holosuite.ipyn`. First script tells which jobs got error. They can be processed in JASMIN again by changing the dedicated (usually commented) part in `reconSomeSequences.py`. Modify the list of bad errors and the amount of jobs for the first run. Otherwise, they can be run again in the PC using `Scripting\useHolosuiteSequence.m`.

#### Copy the results back to PC

Use bash script from MobaXterm in `PC-W:get_results_scp.sh`.

## 5. Tracking

### Prepare config file to get the positions of the particles

Some properties are set automatically when the config files were created, as:
```
Dynamic properties:
    predictor = D:\path\to\whole_tree.mat
    HistMinSize = 1*um
    NameInstrument = Optronis@355nm
    NameCampaign = GlassBeads2um
    IncludeTotal = 1
```

### Run use_wolke

Once the rules and the tree are set, `use_wolke.m` is used for applying the tree to multiple hist files using a specific config file (with required fields in Dynamic Properties). This script gets a classification of the particles as set in the tree, adapts the coordinates to the spherical illumination setup and shows histograms and images of the detected particles.

# 5. Analysis of experimental holograms

## 2021-02-15 : Levitation of droplets (355 nm, Vistek hr120 camera)

Images of a levitated big drop, and levitated droplets at different distances in propagation axis.

### Calibration

Reconstruction of background averaged image at distances `z_autofoc 15e-3 70e-3 5000`, `L 76e-3`, `windows 1.4761 5e-3 1.4761 12e-3`. Result in `C:\Work\Scripting\results\20210215\Indices_FirstLevitation_Calibration.mat`.

It is not easy to observe particles on the sensor-side window.

## 2021-03-10 : Levitation of droplet (355 nm, Optronis camera)

Fast acquisition of levitated small droplet. 9 images at 540 fps

### Calibration

Reconstruction of background averaged image at distances `z_autofoc 5e-3 72e-3 5000`, `L 78e-3`, `windows 1.4761 5e-3 1.4761 12e-3`. Result in `C:\Work\Scripting\results\20210310\Indices_FirstLevitation_Calibration.mat`.

Could be Sapphire window (source-side) WG31050 5 mm @355nm n=1.793; ne=1.7969, no=1.7882

There is a 2 mm window between the sensor and the other window. Very dirty sensor and first sensor-side window. It may add reflections if the coating is not the proper one.

## 2021-03-29 : Levitation of droplet (355 nm, Optronis camera)

See `Experiments in cold room.ipynb`

Big levitated droplet and background images. Cleaner camera.

### Calibration

Reconstruction of background averaged image at distances `z_autofoc 2e-3 71e-3 5000`, `L 78e-3`, `windows 1.793 5e-3 1.4761 12e-3`. Optronis camera requires much less memory (trying 2GB). Result in `C:\Work\Scripting\results\20210329\Indices_Calibration.mat`.

There are no clear positions to extract the location of the windows' interfaces.

### Reconstruction of a single hologram using Holosuite in local PC

The calibration image, equalised and averaged, is reconstructed using `useHolosuiteOneHologram`. The config file is in the same folder as the image.

### Reconstruction of a single hologram using Holosuite in JASMIN

Same as above but using `sbatch slurm_HRSH.sh`. Use git repository in `C:\Work\Scripting\CompileHolosuite`.

## 2021-04-29 : Levitation of droplet

I removed the window in the Optronis camera. The linear stage is now better fixed to the axis of propagation.

- CRM1/M with window. Distance from rotating side to ring (1mm) : 4.33 mm. Fixed side to window : 4.4 mm (sampled volume).
- Distance from CP32/M to CRM1/M (not rotating part) : 12.73 mm (5.39 mm to rotating part)
- Distance from CRM1/M to SM2 tube (ext thread) 33.80 mm
- Device0Strobe : -140 us, exposure 50 us
- Background images, sequence of levitated small droplet, small droplet at different distances with motorised linear stage (seq2-3mmStep, seq3-2mmStep).
- R2L2S3P1 in XYFM1/M, with 2 posts and RA90/M to EO 03655 on linear stage. Without sensor-window and with it. Steps 1mm.

Procedure for placing the camera:
1. Remove window source-side
2. Place the LCP02/M with 3" rods and LCP09.
3. Place camera tube through LCP09. Adjust LCP02/M and LCP09.
4. Adjust camera
5. Remove LCP02/M and LCP09.
6. Place window source-side.

### Calibration with R2L2S3P1

Simulation with `C:\Work\Scripting\PeriodicSample.m` shows that the frequency of the hologram keeps constant when a planar wavefront propagates through a grid.

The script `C:\Work\Scripting\focus_to_camera_estimation_long_set.m` finds the effective source-sensor distance. It can estimate the spacing of the grid and the location of the sample. In the case of having both windows, the effective distance is 87.55 mm, and the physical distance using source-window 5mm n=1.793, sensor-window 12mm n=1.4761, is 93.03 mm. In the case of taking the sensor-window out, the effective distance is 87.97 mm, and the physical distance is 90.18 mm.

### Reconstruction

Reconstruction of first file in Seq1 gives a $z_\mathrm{rec}$ position of reconstruction at 105 mm, using $L_\mathrm{eff} = 87.6$ mm, means $z_\mathrm{eff} = z_\mathrm{rec} L_\mathrm{eff} / (z_\mathrm{rec} + L_\mathrm{eff}) = 47.75$ mm. This is not the real distance if there is a window in the sensor side.

In [7]:
mm = 1e-3
z_rec = 105 *mm
L_eff = 87.6 *mm
print('The effective distance from sensor to particle is ', z_rec*L_eff/(z_rec+L_eff)/mm, 'mm.')

The effective distance from sensor to particle is  47.75700934579439 mm.


#### Seq1

Reconstruction of holograms take around 4550s = 1h15m each. Other times are not significant.

# 2021-05-28 : Calibration verification using 1951 USAF

## Background images

These images were averaged into a single background image using the python script `Mean_Image.py` in the acquisition PC. This script is part of the GitHub project `ZaberAndEuresys`.

## Calibration with R2L2S3P1

The script `C:\Work\Scripting\focus_to_camera_estimation_long_set.m` finds the effective source-sensor distance 88.53 mm (with 2 windows and 1.5 mm soda lime glass, same as USAF) and the real distance (fused silica window 12mm, sapphire window 5mm, 1.5mm SLG) 95.14 mm.

## USAF images

A new filtering process was created for analysing a sequence where the background cannot be estimated from different holograms in the sequence. A background image is used instead.

In [3]:
# Auxiliary maths and scripting

import numpy as np
import os

x = np.floor(np.arange(9)/9*5)
print(x)
id = 9
id_in_seq = np.sum(x[:id] == x[id-1])
print(id_in_seq)

folder = "C:\\Work\\Scripting\\CompileHolosuite\\Results\\20210429\\Seq3"
for f in os.scandir(folder):
    print(f.path)

[0. 0. 1. 1. 2. 2. 3. 3. 4.]
1
C:\Work\Scripting\CompileHolosuite\Results\20210429\Seq3\Seq_end
C:\Work\Scripting\CompileHolosuite\Results\20210429\Seq3\Seq_end_2mm
C:\Work\Scripting\CompileHolosuite\Results\20210429\Seq3\Seq_end_4mm
C:\Work\Scripting\CompileHolosuite\Results\20210429\Seq3\Seq_end_6mm


# 2021-09-02 : Calibration using 1951 USAF target

The USAF target is placed at different positions in $z$ and the geometrical transformation between in focus reconstructions is found. Then, knowing the scalings and the difference of positions of the target for each image, the distance from sensor to source is estimated. The XY position of the source can also be estimated.

The Photron software may deliver images with an histogram. Use `C:\Work\Scripting\remove_hist.m` to remove the histograms from the images.

Use WaldemarPC to run `D:\HoloICE\Scripting\Focus_detection_single_plane.m` to find the z positions for in-focus reconstruction of the USAF target. Use `get_ranges = true` to do a first reconstruction using `K_axial_sampling = 20` may allow getting good ranges to do the autofocusing search with a lower `K_axial_sampling` (perhaps = 4). Reconstruct at those distances to be able to run an image registration algorithm. Obtain the scaling of each distance. Use the Single_ASM git repository that links `c:\Work\Scripting` to `~\Documents\Work\JASMIN\Projects\Single ASM`. Check that git repos are up-to-date in all places.

Run `analysis_Indices_focus_detection.m` to get the geometrical transforms. Run `focus_to_camera_estimation_USAF.m` to obtain the source position.

Information about the 1951 USAF target from Thorlabs: [Link](https://www.thorlabs.com/newgrouppage9.cfm?objectgroup_id=4338)

The estimation is based on the formulae in Optics Maths.ipynb.

# 2021-09-02 : Processing of glass beads sequences of holograms

In the HoloICE PC there are holograms for the 15 and 2 um glass beads in `C:\Users\HoloICE\Documents\Python Scripts\Calibration\Images`.

## 01 - 02 - 03 Triggered acquisition

### 01 and 02

These holograms don't need more processing. They were just to try to trigger the acquisition when something shows up.

### 03 Added beads - Optronis

Images in `HoloICE:C:\Users\HoloICE\Documents\Python Scripts\Calibration\Images\03 - Adding glass beads`

1. Seq01 : no sequences detected with `Particle_Detection_images.py`
2. Seq02 : 4 sequences turned to raw hologram videos. Turned to contrast images.
3. Seq03 : 8 sequences turned to raw hologram videos. Turned to contrast images. No valid sequences.
4. Seq04 : 4 sequences turned to raw hologram videos. Turned to contrast images, with fitted backgrounds.

## 04, 05, 08 Calibration Optronis

The calibration images of the grid moved with motor. A difference in illumination between 04 and 05. A more robust setup with 2" cage system was introduced before calibration 08. Images processed in PC-W.

1. Result from 04: \
The effective distance from the source to the sensor is 66.69 mm (66.16 mm with the grid in). \
The actual distance from the source to the sensor is 68.81 mm.
2. Result from 05: \
The effective distance from the source to the sensor is 67.38 mm (66.85 mm with the grid in). \
The actual distance from the source to the sensor is 69.50 mm.
3. Result from 08: \
The effective distance from the source to the sensor is 57.05 mm (56.53 mm with the grid in). \
The actual distance from the source to the sensor is 59.18 mm.

## 06 07 - Beads 

### 06 Seq1 (using calibration 05)

A lot of movement. Makes it difficult to detect the presence of particles. Use all images. Images in `Laptop:C:\Users\j05625pe\Documents\Work\Holograms\Glass beads\06 - Beads\Seq1`

### 07 Seq (using calibration 08)

Images in `PC-W:D:\HoloICE\Holograms\Glass beads\07 - Beads`

1. Seq1 : A single sequence with a few images. The reconstruction gets a lot of noisy artifacts that hinder the particles. Thi is because of the movement of the sensor with respect to the source. It may be useful to extract the contribution from a particle a long distance away. It make sense using a numsz > 15. I erased the hist files as they were too large.
2. Seq2 : 4 sequences.

## 09 - SVS Vistek - 15 beads

### Calibration

Processed in PC-W: The effective distance from the source to the sensor is 65.04 mm (64.51 mm with the grid in). \
The actual distance from the source to the sensor is 67.16 mm.

### Sequences

Processed in PC-W:
1. Seq1 : frame rate 9.4 fps, laser pulse rate 103 Hz, using NDF OD0.7. The contrast should be simply the difference with the hologram: implemented with PARAMETER JUST_DIF
2. Seq2 : one pulse per frame, 9.4 fps. The parameter GET_BAD_ROWS_ROLLING_SHUTTER is set to True in contrast_video_from_images.py
3. Seq3 : idem 2 but this time with beads!

## Photron 15um

### Calibration USAF

Processed in PC-W. Still without results. The last step is too high and produces errors.

### Sequences

Processed in PC-W. Very long exposure time, it doesn't make sense reconstruction. No contrast images.

## 10 - New beads (2um glass beads)

### Optronis calibration

Processed in Laptop: \
The effective distance from the source to the sensor is 64.64 mm (64.11 mm with the grid in). \
The actual distance from the source to the sensor is 66.76 mm.

### Optronis sequences

Processed in PC-W. Do not consider cols on the right side for detecting particles. 

Seq3 : 14 sequences turned to contrast images.

### SVS-Vistek

In folder `10 - New beads` there are two folders with images from this camera: `SVS Vistek Calibration` and `SVS Vistek 2um`. The calibration is based on the grid sample so it uses the script `C:\Work\Scripting\focus_to_camera_estimation_long_set.m` to get the source to sensor distance. Processed in PC-W:

The effective distance from the source to the sensor is 64.82 mm (64.29 mm with the grid in). \
The actual distance from the source to the sensor is 66.94 mm.

The images are prepared to be used as contrast images by using `Calibration\contrast_video_from_images.py`, which gets a reference hologram from the previous frames (median) and is subtracted and divides the current frame before reconstruction. Using the parameter OUTPUT_IMS in False gets a video for showing purposes (REDUCE_BY = 6 recommended, GLAG_DO_ABS = True)

1. Seq1 : 4 images, 2 beads
2. Seq2 : Several beads.
3. Seq3 : No beads. Erased.
4. Seq4 : Only image 11 with beads.

These images are analysed by Holosuite in JASMIN. See section 4.

## Photron 2um

### 2021-08-12 Calibration USAF

The effective distance from the source to the sensor (with the windows, but without the USAF target) is 95.56 mm. \
The actual distance from the source to the sensor is 97.56 mm.

### 2021-08-12 Sequences

1. Microspheres_2um_01_C001H001S0001: 6000fps, 1/20ks, 16 sequences. No particles. Erased.
2. Microspheres_2um_02_C001H001S0001 : Config same as 1. 3 sequences. Not beads but other stuff.
3. Microspheres_2um_03_C001H001S0001 : Config same as 1. 1 sequence with a large particle and other flying particles. The spray is all smeared.
4. Microspheres_2um_04_C001H001S0001 : 4000 fps, 1/150ks. 3 sequences, smeared.
5. MoreFlux_2um_inv800ks_C001H001S0001 : 4000 fps, 1/800ks. 5 sequences.

### 2021-09-16 Calibration USAF

#### SeqCalib00

Measuring distances with calipers, but it moves the target.
The effective distance from the source to the sensor (with the windows, but without the USAF target) is 54.54 mm.\
The actual distance from the source to the sensor is 56.54 mm.

#### SeqCalib01

The effective distance from the source to the sensor (with the windows, but without the USAF target) is 55.88 mm.\
The actual distance from the source to the sensor is 57.89 mm.

#### SeqCalib02

The effective distance from the source to the sensor (with the windows, but without the USAF target) is 58.59 mm. \
The actual distance from the source to the sensor is 60.60 mm.

### 2021-09-16 Sequences

1. 2um_inv950ks_20210916 : 6000fps, 1/950ks. 6 sequences.
2. 2um_inv1.5Ms_maxpower_C001H001S0001 : 6400fps, 1/1.5Ms. 30 sequences.

- 16/06/2022 Using sequences 18, 22, 24, 29, 31 and 37 to compare detection of particles with standard ASM and SAASM. The config files for those sequences have been updated first for SAAASM. The rules were found by performing a sequence reconstruction of sequence 22 (useHolosuiteSequence.m) and then using carft to hand-classify the particles. The tree was saved. Once that's done, the histograms allowed getting useful rules (artifacts dropped but not particles). Those rules were applied to get the hist files for the other sequences. Then, the saved tree was used and updated in other sequences.
- 17/06/2022 Too many particles with focus below the given minimum distance. JASMIN is not working. Doing ASM and SAASM in Waldemar's. ASM is using the same rules, it will probably need repeating.

# Reconstruction

- Optronis required 2GB per job.
- Done with `Glass Beads\10 - 2um beads\Optronis\Seq3\seq0001`
- 03/11/2021 : Optronis 2um in JASMIN. JOBID 9052549 (244 jobs, 6G memory)
- 11/11/2021 Optronis and SVS 2um contrast images recalculated. Optronis reconstructed and processed up to histogram.
- 12/11/2021 Impossible to see small particles in Photron images with holoViewer, reconstruction in PC-W (Kz 4). Pretty enough resolution. Doing single reconstruction with SVS images and profiling 47-50 mm : 12h (6 cpus, 98% of time in parfor of reconstruction), no particles. 15/11 reconstructing 08 between 27-40 mm (6 cpus).
- 16/11/2021 Prepared a tre for Photron images 16/9 inv1.5Ms.


# Pollen

## 07/10/2021

### Photron: ash pollen

max aperture 0.11

#### 532nm CW

Nozzle at around 4.95 cm from camera. Distance from sensor to camera around 6 cm. The air flow was as low as possible. Video at 6400 fps 1024x1024. Exposure time 333us.

The reference beam is not good. The reconstruction for the calibration using USAF target is too bad.

#### 355nm Pulsed

SYNC OUT, pulse width 1000us. OD1.0 needs to be multiplied by 3. OD0.3 is too bright

Calibration images with 0.25 mm separation. The distance is 69.37 mm.

### 11 - Pollen SVS

max aperture = 0.21

#### Calibration using grid

The effective distance from the source to the sensor is 67.01 mm (66.48 mm with the grid in). \
The actual distance from the source to the sensor is 71.96 mm.

#### Ash_02

Working with some particles. See next python cell to get details.

### 12 - Pollen Optronis

max aperture = 0.1

#### Calibration using grid

The effective distance from the source to the sensor is 65.82 mm (65.30 mm with the grid in).\
The actual distance from the source to the sensor is 70.78 mm.

#### Calibration using USAF 1951

The effective distance from the source to the sensor (with the windows, but without the USAF target) is 58.56 mm.
The actual distance from the source to the sensor is 64.05 mm.

In [11]:
import numpy as np
# Aperture of set-ups
# Optronis
max_dim = 1700 / 2 * 8e-3
L = 65.82
# SVS
max_dim = 13000 / 2 * 2.2e-3
L = 67.01
max_aperture = max_dim / np.sqrt(max_dim**2 + L**2)
print(f'Max aperture : {max_aperture:.2f}')

# Aperture of particles in SVS Ash_02
L = 67.01e-3
# z_asm, x_non_mag, y_non_mag, z_saasm
particles = np.array([[48.6528e-3, -4951e-6, -8942e-6, 28.1873e-3, 3],  # particle 3
                      [48.6915e-3, 1544e-6, -4323e-6, 28.2003e-3, 5],   # particle 5
                      [129.9778e-3, 844e-6, 7928e-6, 44.072e-3, 0],     # particle 0
                      [49.1214e-3, -5664e-6, -587e-6, 28.218e-3, 6],    # particle 6
                      [48.386e-3, 11175e-6, -3883e-6, 28.093e-3, 7],    # particle 7
                      [48.386e-3, 11204e-6, -1681e-6, 28.093e-3, 8],    # particle 8
                      [55.911e-3, -10055e-6, 8456e-6, 30.478e-3, 10]])   # particle 10
for part in particles:
    magnification = L / (L - part[3])
    rho = np.linalg.norm(part[1:3] / magnification)
    sin_theta = rho / np.sqrt(rho**2 + part[3]**2)
    print(f'Particle {int(part[4]):d} with aperture : {sin_theta:.3f}, and magnification {magnification:.2f}X. Scale in figure: {2.2*25*magnification:.2f} um.')

Max aperture : 0.21
Particle 3 with aperture : 0.206, and magnification 1.73X. Scale in figure: 94.93 um.
Particle 5 with aperture : 0.094, and magnification 1.73X. Scale in figure: 94.96 um.
Particle 0 with aperture : 0.062, and magnification 2.92X. Scale in figure: 160.67 um.
Particle 6 with aperture : 0.116, and magnification 1.73X. Scale in figure: 95.01 um.
Particle 7 with aperture : 0.238, and magnification 1.72X. Scale in figure: 94.70 um.
Particle 8 with aperture : 0.228, and magnification 1.72X. Scale in figure: 94.70 um.
Particle 10 with aperture : 0.229, and magnification 1.83X. Scale in figure: 100.89 um.


# Splashing drops

## Calibration using grid

Result from using script `C:\Work\Scripting\focus_to_camera_estimation_variable_steps.m`.

The effective distance from the source to the sensor is 65.82 mm (65.30 mm with the grid in).
The actual distance from the source to the sensor is 67.33 mm.

In [2]:
# Fiber optics time delay
c = 2.998e8  # speed of light m/s
virtual_frame_rate = 100e3
n_fibre = 1.467
c_fibre = c / n_fibre

time_between_pulses = 1 / virtual_frame_rate
length_fibre = c_fibre * time_between_pulses

print(f'The length of a fibre to obtain a delay of {time_between_pulses/1e-9:.2f} ns is {length_fibre:.2} m.')

The length of a fibre to obtain a delay of 10000.00 ns is 2e+03 m.
Max aperture : 0.21


# Using collimated beam

## Calibration

Measuring the pixel size with the R2L2S3P1. An in-focus image is acquired. Use get_main_frequency to get a frequency (between 1 and $\sqrt{2}$), with 1 at sampling rate / 2. The period of R2L2S3P1 is 125 um.

In [1]:
dx = 20e-6
f_from_matlab = 64.6143e-3
magnified_dx = f_from_matlab * 125e-6 / 2
print(f'The size of the effective pixel is {magnified_dx*1e6} um.')
print(f'The magnification of the setup is {dx / magnified_dx:.2f}.')

The size of the effective pixel is 4.03839375 um.
The magnification of the setup is 4.95.
